In [1]:
import numpy as np 
from load_data import load_data
train, valid, test  = load_data('./mnist.pkl.gz')

D:\anaconda\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


... loading data


In [2]:
raw_X_train = train[0]
raw_y_train = train[1]
raw_X_val = valid[0]
raw_y_val = valid[1]
raw_X_test = test[0]
raw_y_test = test[1]

In [3]:
n_train_samples = raw_X_train.shape[0]
n_val_samples = raw_X_val.shape[0]
n_test_samples = raw_X_test.shape[0]

X_train = raw_X_train.reshape(n_train_samples, 28, 28, 1)
X_val = raw_X_val.reshape(n_val_samples, 28, 28, 1)
X_test = raw_X_test.reshape(n_test_samples, 28, 28, 1)

In [4]:
from keras import utils
Y_train = utils.to_categorical(raw_y_train, num_classes=10)
Y_val = utils.to_categorical(raw_y_val, num_classes=10)
Y_test = utils.to_categorical(raw_y_test, num_classes=10)

D:\anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras import Model
from keras import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
def mnist_model():
    X_input = Input(shape=(28, 28, 1), name='input')
    
    X = Conv2D(6, (5, 5), padding='same', kernel_initializer='he_normal', bias_initializer='zeros')(X_input)
    X =Activation('relu')(X)
    X= MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Conv2D(16,(5, 5), padding="same", kernel_initializer='he_normal', bias_initializer='zeros')(X)
    X = Activation("relu")(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Flatten()(X)
    X = Dense(120, kernel_initializer='he_normal', bias_initializer='zeros')(X)
    X = Activation("relu")(X)
    
    X = Dense(84, kernel_initializer='he_normal', bias_initializer='zeros')(X)
    X = Activation("relu")(X)
    
    X = Dense(10)(X)
    X_output = Activation("softmax")(X)
    
    model = Model(inputs = X_input, outputs = X_output)
    
    return model

In [6]:
he_model = mnist_model()

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
from keras.optimizers import SGD
he_model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.001), metrics=["accuracy"])

In [8]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('he_2_training.log')

In [9]:
history = he_model.fit(X_train, Y_train, validation_data=[X_val, Y_val], batch_size=128, epochs=20, verbose=1, callbacks=[csv_logger])

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 61s 1ms/step - loss: 2.0713 - acc: 0.2987 - val_loss: 1.7552 - val_acc: 0.5722
Epoch 2/20
50000/50000 [==============================] - 75s 1ms/step - loss: 1.4368 - acc: 0.6768 - val_loss: 1.0494 - val_acc: 0.7749
Epoch 3/20
50000/50000 [==============================] - 71s 1ms/step - loss: 0.8463 - acc: 0.8012 - val_loss: 0.6216 - val_acc: 0.8483
Epoch 4/20
50000/50000 [==============================] - 59s 1ms/step - loss: 0.5705 - acc: 0.8517 - val_loss: 0.4585 - val_acc: 0.8804
Epoch 5/20
50000/50000 [==============================] - 57s 1ms/step - loss: 0.4552 - acc: 0.8750 - val_loss: 0.3818 - val_acc: 0.8962
Epoch 6/20
50000/50000 [==============================] - 58s 1ms/step - loss: 0.3942 - acc: 0.8900 - val_loss: 0.3379 - val_acc: 0.9064
Epoch 7/20
50000/50000 [==============================] - 59s 1ms/step - loss: 0

In [10]:
(loss, accuracy) = he_model.evaluate(X_test, Y_test, batch_size=128, verbose=1)
print("accuracy: {:.2f}%".format(accuracy * 100))

10000/10000 [==============================] - 6s 579us/step
accuracy: 94.87%


In [11]:
import pandas as pd
df = pd.read_csv('he_2_training.log')
print(df)

    epoch      acc      loss  val_acc  val_loss
0       0  0.29868  2.071327   0.5722  1.755184
1       1  0.67680  1.436822   0.7749  1.049397
2       2  0.80120  0.846293   0.8483  0.621649
3       3  0.85168  0.570493   0.8804  0.458493
4       4  0.87500  0.455233   0.8962  0.381819
5       5  0.89002  0.394170   0.9064  0.337946
6       6  0.89922  0.355545   0.9085  0.315264
7       7  0.90640  0.327789   0.9198  0.287086
8       8  0.91254  0.306760   0.9256  0.269901
9       9  0.91654  0.289127   0.9283  0.256593
10     10  0.92010  0.274704   0.9308  0.244567
11     11  0.92324  0.261713   0.9340  0.235692
12     12  0.92692  0.250821   0.9366  0.225262
13     13  0.92910  0.240508   0.9378  0.219435
14     14  0.93186  0.231221   0.9393  0.212826
15     15  0.93394  0.222993   0.9427  0.203149
16     16  0.93616  0.215650   0.9444  0.195958
17     17  0.93822  0.208849   0.9457  0.191965
18     18  0.94002  0.202722   0.9476  0.185919
19     19  0.94066  0.196641   0.9489  0